In [31]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, Point
import ast

# import dfs
buildings_df = pd.read_csv('Processed_Building_Footprints.csv')
ground_df = pd.read_csv("Training_data_uhi_index_UHI2025-v2.csv")

# clean dfs
buildings_df = buildings_df.drop(columns = ['path', 'id', 'tessellate', 'extrude', 'visibility'])


# Test

In [32]:
buildings_df['coordinates'] = buildings_df['coordinates'].apply(ast.literal_eval)
buildings_df['geometry'] = buildings_df['coordinates'].apply(Polygon)

gdf_buildings = gpd.GeoDataFrame(buildings_df, geometry='geometry', crs="EPSG:4326")

gdf_ground = gpd.GeoDataFrame(
    ground_df,
    geometry=gpd.points_from_xy(ground_df['Longitude'], ground_df['Latitude']),
    crs="EPSG:4326"
)


gdf_buildings = gdf_buildings.to_crs("EPSG:32618")
gdf_ground = gdf_ground.to_crs("EPSG:32618")

def calculate_building_area(point, buildings, buffer_size):
    buffer = point.buffer(buffer_size)
    intersecting_buildings = buildings[buildings.intersects(buffer)]
    total_area = intersecting_buildings.geometry.intersection(buffer).area.sum()
    return total_area

gdf_ground['building_area_25']  = gdf_ground.geometry.apply(lambda pt: calculate_building_area(pt, gdf_buildings, 25))
gdf_ground['building_area_50']  = gdf_ground.geometry.apply(lambda pt: calculate_building_area(pt, gdf_buildings, 50))
gdf_ground['building_area_100'] = gdf_ground.geometry.apply(lambda pt: calculate_building_area(pt, gdf_buildings, 100))

# Optionally, convert back to a pandas DataFrame if you don't need the geometry column:
result_df = pd.DataFrame(gdf_ground.drop(columns='geometry'))

result_df.head()


   Longitude   Latitude          datetime  UHI Index  building_area_25  \
0 -73.909167  40.813107  24-07-2021 15:53   1.030289        183.978594   
1 -73.909187  40.813045  24-07-2021 15:53   1.030289        266.930647   
2 -73.909215  40.812978  24-07-2021 15:53   1.023798        321.122238   
3 -73.909242  40.812908  24-07-2021 15:53   1.023798        343.873068   
4 -73.909257  40.812845  24-07-2021 15:53   1.021634        382.402733   

   building_area_50  building_area_100  
0       1597.323120       11046.690248  
1       1729.473614       10814.226850  
2       1951.435804       10525.136565  
3       2287.039351       10446.113598  
4       2652.734741       10479.891453  


In [34]:
result_df.to_csv("Building_Area.csv", index = False)